In [2]:
import tensorflow as tf


In [5]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-1.1.4-cp37-cp37m-win_amd64.whl (8.7 MB)
  Using cached pytz-2020.4-py2.py3-none-any.whl (509 kB)


In [7]:
import pandas as pd


In [8]:
df = pd.read_csv(r"C:\Users\User\pliki_csv_do_deep_learning\train.csv")

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df = df.dropna()

In [11]:
### take only independant features
X = df.drop('label', axis=1)

In [12]:
y= df['label']

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
tf.__version__

'2.1.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
voc_size = 5000

## One_hot representation

In [18]:
messages = X.copy()

In [19]:
messages.reset_index(inplace=True)

In [25]:
pip install nltk

  Using cached joblib-0.17.0-py3-none-any.whl (301 kB)
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434677 sha256=680eac962147bf931b7a12636d306b9894d4ca3aa155762356fa5b60804d9e96
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Note: you may need to restart the kernel to use updated packages.Successfully built nltk



In [26]:
import nltk

In [29]:
import re
from nltk.corpus import stopwords

In [32]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', " ", messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [33]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[591, 2617, 1477, 296, 12, 4916, 2673, 2053, 3585, 3979],
 [398, 2122, 4651, 3862, 3505, 4922, 2131],
 [1496, 596, 4851, 920],
 [1162, 1940, 4964, 1458, 4889, 1636],
 [4205, 3505, 513, 1779, 4382, 4146, 3505, 378, 148, 4288],
 [4382,
  136,
  3455,
  328,
  3898,
  1605,
  160,
  3896,
  731,
  967,
  4904,
  4763,
  267,
  734,
  2131],
 [3768, 2742, 4621, 4778, 504, 1528, 1233, 4714, 3872, 133, 4626],
 [838, 4844, 36, 2064, 3719, 2437, 1605, 3780, 3872, 133, 4626],
 [4471, 49, 4971, 2907, 1826, 4120, 3597, 2012, 1605, 4105],
 [2633, 2260, 4553, 3863, 4812, 4558, 806, 2041],
 [2024, 723, 4903, 3501, 4455, 1342, 855, 4949, 1356, 1887, 475],
 [1458, 35, 12, 4120, 1605, 3719],
 [712, 299, 2321, 4944, 3810, 763, 2867, 62, 324],
 [3210, 2497, 4722, 4825, 2587, 259, 2973, 3872, 133, 4626],
 [4456, 3776, 3946, 4468, 1106, 3872, 133, 4626],
 [3546, 418, 2705, 885, 3865, 3032, 4881, 4592, 3945, 3167],
 [4610, 2051, 2122],
 [3987, 3558, 1342, 3344, 1605, 1053, 3236, 2131],
 [923, 938, 4651, 37

### Embeding representation

In [34]:
# make the sentences of fixed length
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

In [36]:
embedded_docs[0] # this is our input to embeddindg layer

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  591,
       2617, 1477,  296,   12, 4916, 2673, 2053, 3585, 3979])

In [40]:
# creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100)) # one layer with 100 neurons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
import numpy as np

In [43]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [44]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [49]:
pip install sklearn

Processing c:\users\user\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e\sklearn-0.0-py2.py3-none-any.whlNote: you may need to restart the kernel to use updated packages.

  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state=42)

### Model training

In [54]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size = 64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 7s 535us/sample - loss: 0.3326 - accuracy: 0.8395 - val_loss: 0.1920 - val_accuracy: 0.9195
Epoch 2/10
12250/12250 [==============================] - 4s 335us/sample - loss: 0.1422 - accuracy: 0.9450 - val_loss: 0.1934 - val_accuracy: 0.9198
Epoch 3/10
12250/12250 [==============================] - 4s 341us/sample - loss: 0.0977 - accuracy: 0.9628 - val_loss: 0.2216 - val_accuracy: 0.9228
Epoch 4/10
12250/12250 [==============================] - 4s 338us/sample - loss: 0.0689 - accuracy: 0.9751 - val_loss: 0.2536 - val_accuracy: 0.9188
Epoch 5/10
12250/12250 [==============================] - 4s 345us/sample - loss: 0.0457 - accuracy: 0.9849 - val_loss: 0.2809 - val_accuracy: 0.9191
Epoch 6/10
12250/12250 [==============================] - 4s 331us/sample - loss: 0.0351 - accuracy: 0.9884 - val_loss: 0.3710 - val_accuracy: 0.9128
Epoch 7/10
12250/12250 [===========================

### Performance Metrics and accuracy

In [55]:
y_pred = model.predict_classes(X_test)


In [59]:
from sklearn.metrics import confusion_matrix 

In [60]:
confusion_matrix(y_test, y_pred)

array([[3166,  253],
       [ 308, 2308]], dtype=int64)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9070422535211268